In [1]:
import torch
import torch.nn as nn

from functools import partial
from dataclasses import dataclass
from collections import OrderedDict

# ResNet
Today we are going to implement the famouse ResNet.
![alt](./images/resnet34.png)
![alt](./images/residual.png)

## Basic Block

PyTorch does not have the 'auto' padding in Conv2d, so we have to code ourself!

In [2]:
class Conv2dAuto(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding =  (self.kernel_size[0] // 2, self.kernel_size[1] // 2) # dynamic add padding based on the kernel_size
        
conv3x3 = partial(Conv2dAuto, kernel_size=3)      
        

In [3]:
conv = conv3x3(in_channels=32, out_channels=64)
print(conv)
del conv

Conv2dAuto(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


Next, we use `ModuleDict` to create a dictionary with different activation functions, this will be handy later.

In [4]:
def activation_func(activation):
    return  nn.ModuleDict([
        ['relu', nn.ReLU(inplace=True)],
        ['leaky_relu', nn.LeakyReLU(negative_slope=0.01, inplace=True)],
        ['selu', nn.SELU(inplace=True)],
        ['none', nn.Identity()]
    ])[activation]


## Residual Block
The residual block takes an input with `in_channels`, applies some blocks of convolutional layers to reduce it to `out_channels` and sum it up to the original input. If their sizes mismatch, then the input goes into an `identity`. We can abstract this process and create a interface.

In [209]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, activation='relu'):
        super().__init__()
        self.in_channels, self.out_channels, self.activation = in_channels, out_channels, activation
        self.blocks = nn.Identity()
        self.activate = activation_func(activation)
        self.shortcut = nn.Identity()   
    
    def forward(self, x):
        residual = self.shortcut(x)
        x = self.blocks(x)
        x += residual
        x = self.activate(x)
        
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels

In [210]:
ResidualBlock(32, 64)

ResidualBlock(
  (blocks): Identity()
  (activate): ReLU(inplace)
  (shortcut): Identity()
)

In [211]:
dummy = torch.ones((1, 32, 1, 1))

block = ResidualBlock(32, 64)
block(dummy)

tensor([[[[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]],

         [[2.]]]])

In ResNet each block has a expansion parameter in order to increase the `out_channels`. Also, the identity is just a Convolution followed by an Activation layer Then, we can implement it and define the `identity` function and

In [212]:
class ResNetResidualBlock(ResidualBlock):
    expansion = 1
    def __init__(self, in_channels, out_channels, expansion=1, downsampling=1, conv=conv3x3, *args, **kwargs):
        super().__init__(in_channels, out_channels)
        self.expansion, self.downsampling, self.conv = expansion, downsampling, conv
        self.shortcut = nn.Sequential(
            nn.Conv2d(self.in_channels, self.expanded_channels, kernel_size=1,
                      stride=self.downsampling, bias=False),
            nn.BatchNorm2d(self.expanded_channels)) if self.should_apply_shortcut else nn.Identity()
        
        
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.expanded_channels

In [213]:
ResNetResidualBlock(32, 64)

ResNetResidualBlock(
  (blocks): Identity()
  (activate): ReLU(inplace)
  (shortcut): Sequential(
    (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

### Basic Block

In [214]:
def conv_bn(in_channels, out_channels, conv, *args, **kwargs):
    return nn.Sequential(conv(in_channels, out_channels, *args, **kwargs), nn.BatchNorm2d(out_channels))

In [251]:
class ResNetBasicBlock(ResNetResidualBlock):
    expansion = 1
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.blocks = nn.Sequential(
            conv_bn(self.in_channels, self.out_channels, conv=self.conv, bias=False, stride=self.downsampling),
            activation_func(self.activation),
            conv_bn(self.out_channels, self.expanded_channels, conv=self.conv, bias=False),
        )
    

In [252]:
dummy = torch.ones((1, 32, 10, 10))

block = ResNetBasicBlock(32, 64)
block(dummy).shape
print(block)

ResNetBasicBlock(
  (blocks): Sequential(
    (0): Sequential(
      (0): Conv2dAuto(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ReLU(inplace)
    (2): Sequential(
      (0): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (activate): ReLU(inplace)
  (shortcut): Sequential(
    (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


### BottleNeck
Now we can define the bottleneck

In [238]:
class ResNetBottleNeckBlock(ResNetResidualBlock):
    expansion = 4
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, expansion=4, *args, **kwargs)
        self.blocks = nn.Sequential(
           conv_bn(self.in_channels, self.out_channels, self.conv, kernel_size=1),
             activation_func(self.activation),
             conv_bn(self.out_channels, self.out_channels, self.conv, kernel_size=3, stride=self.downsampling),
             activation_func(self.activation),
             conv_bn(self.out_channels, self.expanded_channels, self.conv, kernel_size=1),
        )
    

In [239]:
dummy = torch.ones((1, 32, 10, 10))

block = ResNetBottleNeckBlock(32, 64)
block(dummy).shape
print(block)

ResNetBottleNeckBlock(
  (blocks): Sequential(
    (0): Sequential(
      (0): Conv2dAuto(32, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ReLU(inplace)
    (2): Sequential(
      (0): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): ReLU(inplace)
    (4): Sequential(
      (0): Conv2dAuto(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (activate): ReLU(inplace)
  (shortcut): Sequential(
    (0): Conv2d(32, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


### Layer

In [257]:
class ResNetLayer(nn.Module):
    def __init__(self, in_channels, out_channels, block=ResNetBasicBlock, n=1, *args, **kwargs):
        super().__init__()
        # 'We perform downsampling directly by convolutional layers that have a stride of 2.'
        downsampling = 2 if in_channels != out_channels else 1
        
        self.blocks = nn.ModuleList([
            block(in_channels , out_channels, *args, **kwargs, downsampling=downsampling),
            *[block(out_channels * block.expansion, 
                    out_channels, downsampling=1, *args, **kwargs) for _ in range(n - 1)]
        ])

    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        return x

In [241]:
dummy = torch.ones((1, 32, 48, 48))

layer = ResNetLayer(64, 128, block=ResNetBasicBlock, n=3)
# layer(dummy).shape
layer

ResNetLayer(
  (blocks): ModuleList(
    (0): ResNetBasicBlock(
      (blocks): Sequential(
        (0): Sequential(
          (0): Conv2dAuto(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): ReLU(inplace)
        (2): Sequential(
          (0): Conv2dAuto(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (activate): ReLU(inplace)
      (shortcut): Sequential(
        (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ResNetBasicBlock(
      (blocks): Sequential(
        (0): Sequential(
          (0): Conv2dAuto(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), b

### Encoder

In [229]:
def layer_of_incremeneting_features(in_channels, block=nn.Conv2d, increment=2, n=2, *args, **kwargs):
        blocks = []
        
        for _ in range(n):
            out_channels = in_channels * increment
            blocks.append(block(in_channels, out_channels,  *args, **kwargs))
            in_channels = out_channels
            
        return nn.ModuleList(blocks)
 

In [264]:
class ResNetEncoder(nn.Module):
    """
    ResNet encoder composed by increasing different layers with increasing features.
    """
    def __init__(self, in_channels=3, blocks_sizes=[64, 128, 256, 512], deepths=[2,2,2,2], 
                 activation='relu',*args, **kwargs):
        super().__init__()
        
        self.blocks_sizes = blocks_sizes
        
        self.gate = nn.Sequential(
            nn.Conv2d(in_channels, self.blocks_sizes[0], kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(self.blocks_sizes[0]),
            activation_func(activation),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.in_out_block_sizes = list(zip(blocks_sizes, blocks_sizes[1:]))
        self.blocks = nn.ModuleList([ 
            ResNetLayer(blocks_sizes[0], blocks_sizes[0], n=deepths[0], activation=activation, *args, **kwargs),
            *[ResNetLayer(in_channels * block.expansion, 
                          out_channels, n=n, activation=activation, *args, **kwargs) 
              for (in_channels, out_channels), n in zip(self.in_out_block_sizes, deepths[1:])]       
        ])
        
        
    def forward(self, x):
        x = self.gate(x)
        for block in self.blocks:
            x = block(x)
        return x

In [265]:
class ResnetDecoder(nn.Module):
    """
    This class represents the tail of ResNet. It performs a global pooling and maps the output to the
    correct class by using a fully connected layer.
    """

    def __init__(self, in_features, n_classes):
        super().__init__()
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.decoder = nn.Linear(in_features, n_classes)

    def forward(self, x):
        x = self.avg(x)
        x = x.view(x.size(0), -1)
        x = self.decoder(x)
        return x


In [266]:
class ResNet(nn.Module):
    
    def __init__(self, in_channels, n_classes, *args, **kwargs):
        super().__init__()
        self.encoder = ResNetEncoder(in_channels, *args, **kwargs)
        self.decoder = ResnetDecoder(self.encoder.blocks[-1].blocks[-1].expanded_channels, n_classes)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [267]:
from torchsummary import summary

summary(ResNet(3, n_classes=1000, block=ResNetBasicBlock, deepths=[3, 4, 6, 3]).cuda(), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
          Identity-5           [-1, 64, 56, 56]               0
        Conv2dAuto-6           [-1, 64, 56, 56]          36,864
       BatchNorm2d-7           [-1, 64, 56, 56]             128
              ReLU-8           [-1, 64, 56, 56]               0
        Conv2dAuto-9           [-1, 64, 56, 56]          36,864
      BatchNorm2d-10           [-1, 64, 56, 56]             128
             ReLU-11           [-1, 64, 56, 56]               0
 ResNetBasicBlock-12           [-1, 64, 56, 56]               0
         Identity-13           [-1, 64, 56, 56]               0
       Conv2dAuto-14           [-1, 64,

In [259]:
from torchvision.models import resnet101, resnet34, resnet18

# resnet101(False)

summary(resnet34(False).cuda(), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [128]:

dummy = torch.ones((1,32, 12, 12))

ResidualBlock(ResNetBottleNeckBlock(32, 64), identity=ResNetIdentity(32, 64))

NameError: name 'ResNetIdentity' is not defined

In [60]:
BottleNeckBlock(32, 64)

BottleNeckBlock(
  (act): ReLU(inplace)
  (blocks): Sequential(
    (0): Sequential(
      (0): Conv2dAuto(32, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ReLU(inplace)
    (2): Sequential(
      (0): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): ReLU(inplace)
    (4): Sequential(
      (0): Conv2dAuto(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (identity): Sequential(
    (0): Conv2d(32, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [90]:
class SEModule(nn.Module):
    """
    Squeeze and Excitation module https://arxiv.org/abs/1709.01507
    """

    def __init__(self, n_features, ratio=8, *args, **kwargs):
        super().__init__()

        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        self.se = nn.Sequential(
            nn.Linear(n_features, n_features // ratio),
            nn.ReLU(inplace=True),
            nn.Linear(n_features // ratio, n_features),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        out = self.avg_pool(x).view(b, c)  # flat
        out = self.se(out).view(b, c, 1, 1)

        return x * out
  

In [193]:
def layer_of_incremeneting_features(in_channels, block=nn.Conv2d, increment=2, n=2, *args, **kwargs):
        blocks = []
        
        for _ in range(n):
            out_channels = in_channels * increment
            blocks.append(block(in_channels, out_channels,  *args, **kwargs))
            in_channels = out_channels
            
        return nn.ModuleList(blocks)
 

In [194]:
layer_of_incremeneting_features(32, kernel_size=3)

ModuleList(
  (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
)

In [7]:
class ResNetBasicBlock(nn.Module):

    def __init__(self, in_channels, out_channels, conv=conv3x3, 
                 activation='relu', downsampling=1, expansion=1, *args, **kwargs):
        super().__init__()
        self.in_channels, self.out_channels, self.conv, self.downsampling = in_channels, out_channels, conv, downsampling
        self.expansion = expansion
        
        self.act = activation_func(activation)
        
        self.blocks = nn.Sequential(OrderedDict({
            'conv1': conv(self.in_channels, self.out_channels, bias=False, stride=downsampling),
            'bnn1': nn.BatchNorm2d(out_channels),
            'act1': self.act, 
            'conv2': conv(self.out_channels, self.expanded_channels,  bias=False),
            'bnn2': nn.BatchNorm2d(self.expanded_channels)
        }))
        
        self.shortcut = self.get_shortcut() if self.should_apply_shortcut else nn.Identity()
        

    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.expanded_channels
    
    def forward(self, x):
        residual = self.shortcut(x)
        x = self.blocks(x)
        x += residual
        x = self.act(x)
        
        return x
        
    def get_shortcut(self):
        return nn.Sequential(
            nn.Conv2d(self.in_channels, self.expanded_channels, kernel_size=1,
                      stride=self.downsampling, bias=False),
            nn.BatchNorm2d(self.expanded_channels),



class Layer(nn.Module):
    def __init__(self, in_channels, out_channels, block=nn.Conv2d, n=1, increment=1, *args, **kwargs):
        super().__init__()
        
        blocks = [block(in_channels, out_channels,  *args, **kwargs)]
        last_out_channels = out_channels
        
        for i in range(1, n):
            temp = last_out_channels * increment
            blocks.append(block(last_out_channels, temp,  *args, **kwargs))
            last_out_channels = temp 
            
        self.blocks = nn.ModuleList(blocks)
        
    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        return x
    
class ResNetLayer(nn.Module):
    def __init__(self,n_channels, out_channels, block=ResNetBasicBlock, n=1, *args, **kwargs):
        super().__init__()
        # 'We perform downsampling directly by convolutional layers that have a stride of 2.'
        self.blocks = nn.ModuleList([
            block(n_channels, out_channels, *args, **kwargs, downsampling=2),
            *[block(out_channels * block.expansion, out_channels, *args, **kwargs) for _ in range(n - 1)]
        ])

    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        return x

In [8]:
b = BottleNeckBlock(32, 64)

b

BottleNeckBlock(
  (act): ReLU(inplace)
  (blocks): Sequential(
    (0): Sequential(
      (0): Conv2dAuto(32, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
    )
    (1): Sequential(
      (0): Conv2dAuto(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
    )
    (2): Sequential(
      (0): Conv2dAuto(32, 256, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
    )
  )
  (shortcut): Sequential(
    (0): Conv2d(32, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [9]:
l = Layer(32, 64, block = partial(ResNetLayer, n=2), n=2, increment=2)

l(dummy).shape
l

AttributeError: type object 'ResNetBasicBlock' has no attribute 'expansion'

In [192]:
dummy = torch.ones((1,32, 12, 12))
block = ResNetBasicBlock(32, 64, downsampling=2)
block(dummy).shape

torch.Size([1, 64, 6, 6])

In [188]:
layer = Layer(32, 64, n=2, increment=2, block=partial(Layer, n=4, kernel_size=3, block=Conv2dAuto))

dummy = torch.ones((1,32, 12, 12))
layer(dummy).shape

torch.Size([1, 128, 12, 12])

In [175]:
x

AttributeError: 'ResNetBasicBlock' object has no attribute 'downsampling'

In [110]:
layer = ResNetLayer(32, 64, n=2)
layer

ResNetLayer(
  (blocks): ModuleList(
    (0): ResNetBasicBlock(
      (act): ReLU(inplace)
      (blocks): Sequential(
        (conv1): Conv2dAuto(32, 64, kernel_size=(3, 3), stride=2, padding=(1, 1), bias=False)
        (bnn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace)
        (conv2): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bnn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Identity()
    )
    (1): ResNetBasicBlock(
      (act): ReLU(inplace)
      (blocks): Sequential(
        (conv1): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bnn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace)
        (conv2): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bnn2): B

In [7]:
dummy = torch.ones((1,32, 12, 12))

In [8]:
basic = BasicBlock(in_channels=32, out_channels=64)
basic(dummy)

RuntimeError: The size of tensor a (12) must match the size of tensor b (6) at non-singleton dimension 3

In [252]:
from torchvision.models import resnet18

In [253]:
resnet18(False)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co